In [1]:
import mysql.connector
import pandas as pd
import random as rand

In [2]:
mydb = mysql.connector.connect( host = 'localhost',
                                user = 'root',
                                database = 'school_library'
                               )

In [3]:
mycursor = mydb.cursor(buffered=True)

In [4]:
mycursor.execute('SELECT * FROM app_user')
app_user = mycursor.fetchall()

In [5]:
df = pd.read_csv('../archive/Books_rating.csv')

In [6]:
df.rename(columns = {'review/score' : 'score',
                    'review/text' : 'text',
                    'review/summary' : 'summary'},
                     inplace = True)
df = df.drop(columns = ['Title', 'Price', 'User_id',
                        'profileName', 'review/time',
                        'review/helpfulness'])
df['score'] = df['score'].astype(int)
df = df.dropna()

In [7]:
def cleanText(text):
    text = text.replace(u'\xa0', u'')
    text = text.replace(u"'", u'')
    text = text.lstrip('\n')
    text = text.lstrip('"')
    text = text.rstrip('\n')
    text = text.rstrip(';')
    text = text.rstrip('"')
    text = text.replace('""', '"')
    text = text.replace(';"', '')
    text = text.replace('";', '')
    text = text.replace('&amp"', '&')
    text = text.replace('"', '')
    return text

In [9]:
with open('rating_insertions.sql', 'a') as f:
    
    for user in app_user[6:]:

        mycursor.execute(f'SELECT school FROM app_user WHERE app_user.id = {user[0]};')
        school = mycursor.fetchone()[0]

        find_book_in_school_query = f'SELECT DISTINCT book_id\
                                    FROM book_copies_per_school\
                                    WHERE school_id = {school};'

        mycursor.execute(find_book_in_school_query)
        books = mycursor.fetchall()

        for i in range(rand.randint(0, 10)):
            sample_row = df.sample()
            score = sample_row['score'].values[0]
            text = cleanText(sample_row['summary'].values[0])
            df = df.drop(sample_row.index)
            f.write(f'INSERT INTO book_rating (book_id, app_user_id, rating, comments, is_published) VALUES ({rand.choice(books)[0]}, {user[0]}, "{score}", "{text}", 1);\n')